In [26]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import os
import os.path
from os.path import join
import numpy as np
import imodelsx
from tqdm import tqdm
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
import data
files_dict = data.load_files_dict_single_site()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Kevin_AnnArbor_LosAngeles_SaltLakeCity
Jonathan_Indianapolis_Milwaukee_Pittsburgh


/home/chansingh/imodelsx/.venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Katie_Dallas_Seattle_Tucson


/home/chansingh/imodelsx/.venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Spencer_Denver_Sacramento
Patty_Charlotte_Houston_Portland
Corina_Buffalo_Cincinnati_SanFrancisco
Manish_Atlanta_Columbus_WashingtonDC
success: 0
fail: ['Los Angeles COMPLETE', 'Salt Lake City', 'Ann Arbor', 'Pittsburgh', 'Milwaukee', 'Indianapolis', 'Tucson', 'Dallas', 'Seattle', 'Denver', 'Sacramento', 'Houston', 'Charlotte', 'Portland', 'Cincinnati', 'San Francisco', 'Buffalo', 'Columbus', 'WashingtonDC', 'Atlanta']


/home/chansingh/imodelsx/.venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


### Filter data for single-site analysis

In [32]:
k = 'Atlanta'
# k = 'Columbus'
df = files_dict[k]
theme_index = np.where(
    np.array(list(map(str.lower, df.columns.values))) == 'theme')[0][0]
col_vals = df.columns[4: theme_index]

# separate into relevant pieces
qs = df['Subcategory']
responses_df = df[col_vals]
themes_df = df[df.columns[theme_index:]]

In [33]:
question, responses, theme_dict = data.get_data_for_question_single_site(
    question_num=0, qs=qs, responses_df=responses_df, themes_df=themes_df)

In [78]:
def numbered_list(classes):
    # return '\n'.join([f'Class {i+1}. {c}' for i, c in enumerate(classes)])
    return '\n'.join([f'{i+1}. {c}' for i, c in enumerate(classes)])


classification_prompt = '''### You are given a question, response, and a numbered list of themes below.

Question: {question}

Response: {response}

Themes:
{classes_as_numbered_list}

### Which of the themes does the response above belong to? Return the theme number.

Answer:'''

llm = imodelsx.llm.get_llm('gpt-4', repeat_delay=3)

In [79]:
if len(theme_dict) > 1:
    print(classification_prompt.format(
        question=question.strip(),
        classes_as_numbered_list=numbered_list(theme_dict.keys()),
        response=responses[0].strip()
    ))

### You are given a question, response, and a numbered list of themes below.

Question: Tell me a little about your experience with seizures

Response: Twin sons with epilepsy

Themes:
1. Child has them only
2. Parent and child both

### Which of the themes does the response above belong to? Return the theme number.

Answer:


In [77]:
llm(classification_prompt.format(
    question=question.strip(),
    classes_as_numbered_list=numbered_list(theme_dict.keys()),
    response=responses[0].strip()
))

not cached


'1'

In [83]:
# check theme_dict validity
for question_num in tqdm(range(len(qs)), position=0):

    question, responses, theme_dict = data.get_data_for_question_single_site(
        question_num=question_num, qs=qs, responses_df=responses_df, themes_df=themes_df)

    if len(theme_dict) > 1:

        assert sum(theme_dict.values()) == sum(responses.notna(
        )), f'question_num: {question_num}, theme_dict: {theme_dict}, responses: {responses}'

 17%|█▋        | 8/46 [00:00<00:00, 1817.09it/s]


AssertionError: question_num: 8, theme_dict: {'Medical professionals': 3, 'Spouse': 3, 'Spouse or other close family': 2, 'Would not consent': 1}, responses: (site code)-001                     NaN
(site code)-002           Professionals
(site code)-003                      No
(site code)-004                     NaN
(site code)-005             Yes- spouse
(site code)-006    Medical professional
(site code)-007                     NaN
(site code)-008          Family members
(site code)-009                  Parent
(site code)-010     Spouse, grandparent
Unnamed: 14                         NaN
Name: 8, dtype: object

In [81]:
responses

(site code)-001       Twin sons with epilepsy
(site code)-002                      Son once
(site code)-003                           Son
(site code)-004                 Son has had 2
(site code)-005     Daughter genetic syndrome
(site code)-006      Self and son with fevers
(site code)-007                Self and child
(site code)-008    Daughter, febrile seizures
(site code)-009                      Daughter
(site code)-010                      Daughter
Unnamed: 14                               NaN
Name: 0, dtype: object

### Run classification

In [ ]:
num_questions = 3
classifications = defaultdict(list)
for question_num in tqdm(range(num_questions), position=0):

    question, responses, theme_dict = data.get_data_for_question_single_site(
        question_num=question_num, qs=qs, responses_df=responses_df, themes_df=themes_df)

    # only do this if there are multiple themes
    if len(theme_dict) > 1:

        for response_num in tqdm(range(len(responses)), position=1):
            response = responses.values[response_num]

            if pd.isna(response):
                classifications[question_num].append(np.nan)
            else:
                prompt = sentiment_prompt.format(
                    question=question.strip(), response=response.strip())
                sentiments[question_num].append(llm(prompt))